## Ball Picking Challenge

### Part 2: Pick and Place

The aim is to program the robot for the second part of the challenge: pick up the balls and transport them to the corner. To do so, you need to reuse the abilities learnt in week 4; please feel free to reuse the code of those notebooks and exercises.

In [1]:
import packages.initialization
import packages.pioneer3dx as p3dx
p3dx.init()

Found ROS controller /pioneer3dx_2770_ip_172_31_43_194


In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import cv2
import numpy

In [4]:
# function to detect blue blobs
def blue_balls():
    hsv = cv2.cvtColor(p3dx.image, cv2.COLOR_RGB2HSV)
    lower_blue = numpy.array([110,  50,  50])
    upper_blue = numpy.array([130, 255, 255])
    mask = cv2.inRange(hsv, lower_blue, upper_blue)
    params = cv2.SimpleBlobDetector_Params()
    params.filterByArea = True
    params.minArea = 5
    detector = cv2.SimpleBlobDetector(params)
    reversemask = 255-mask
    keypoints = detector.detect(reversemask)
    l = []
    for i,kp in enumerate(keypoints):
        l.append((kp.size,) + kp.pt)
    l.sort(key=lambda tup: tup[0], reverse=True)
    return l

In [5]:
# function to detect the red target
def color_blob():
    lower = numpy.array([ 170, 100, 100])
    upper = numpy.array([ 180, 255, 255])
    hsv = cv2.cvtColor(p3dx.image, cv2.COLOR_RGB2HSV)
    mask = cv2.inRange(hsv, lower, upper)
    M = cv2.moments(mask)
    area = M['m00']
    if area > 0:
        cx = int(M['m10']/area)
        cy = int(M['m01']/area)
    else:
        cx = None
        cy = None
    return area, cx, cy

In [6]:
# to check if the ball is centered
def is_ball_centered(cx,cy):
    
    if cx >= 70 and cx < 80:
        return True
    else:
        return False

In [7]:
# to check if the target is centered
def is_blob_centered():
    area, cx, cy = color_blob()
    if area > 0 and cx >= 70 and cx < 80:
        return True
        #print("X: %r Y: %r Area: %r" % (cx,cy,area))
    else:
        return False

In [8]:
# check if the target is close enough
def is_blob_close(tcy):
    area, cx, cy = color_blob()
    #print("Target Y: %r" % cy)
    if area > 0 and cy >= tcy:
        return True
        #print("Final Y: %r" % cy)
    else:
        return False

In [9]:
# function to move towards the target and align
def center_approach(vel,cy):
    while not is_blob_centered():
        p3dx.move(-0.5,0.5)
    p3dx.stop()
    print("---target centered")
    p3dx.sleep(5)
    print("---approaching target")
    while not is_blob_close(cy):
        p3dx.move(vel,vel)
    p3dx.stop()


In [10]:
# function to get individual balls
def get_ball(cx,cy):
    print("getting ball..")
    p3dx.gripper(0.05,0.1)
    #p3dx.tilt(-0.35)
    # align the ball
    while not is_ball_centered(cx,cy):
        p3dx.move(-0.5,0.5)
        b = blue_balls()
        for ball in b:
            cx = ball[1]
            cy = ball[2]
    p3dx.stop()
    p3dx.tilt(-0.47)
    print("done aligning.approaching ball")
    # move towards the balll
    while not cy >= 70:
        p3dx.move(2,2)
        b = blue_balls()
        for ball in b:
            cx = ball[1]
            cy = ball[2]
            #print("Y: %f "%(cy))
    p3dx.stop()
    print("done approaching.getting closer")
    # get closer to ball to pickup
    while not is_ball_centered(cx,cy):
        p3dx.move(-0.5,0.5)
        b = blue_balls()
        for ball in b:
            cx = ball[1]
            cy = ball[2]
            #print("X: %f "%(cx))
    p3dx.stop()
    p3dx.sleep(5)
    while not cy >= 80:
        #print("Y: %f "%(cy))
        prevcy = cy
        p3dx.move(0.2,0.2)
        b = blue_balls()
        for ball in b:
            cy = ball[2]
    # to eliminate the blid spot of kinect when the ball is too close to segment
    p3dx.move(0.2,0.2)
    p3dx.sleep(6)  
    p3dx.stop()

In [11]:
# function to grip the ball
def grip_ball():
    print("gripping ball..")
    p3dx.gripper(0.05,0.0)
    p3dx.sleep(5)
    p3dx.gripper(-0.05,0.0)

In [12]:
# function to move towards the target
def get_target():
    print("getting target..")
    p3dx.tilt(0.0)
    center_approach(2,80)
    p3dx.tilt(-0.47)
    p3dx.sleep(3)
    print("getting closer to target..")
    center_approach(0.5,37)
    #print("test move..")
    # to eliminate the blid spot of kinect when the target is too close to segment
    p3dx.move(0.3,0.3)
    p3dx.sleep(5)
    p3dx.move(0,0)

In [13]:
# function to place the ball and move backwards and turn
def place_ball():
    print("Placing ball..")
    p3dx.gripper(0.05,0.0)
    p3dx.sleep(2)
    p3dx.gripper(0.05,0.1)
    p3dx.sleep(1)
    p3dx.gripper(-0.05,0.1)
    print("moving back")
    # move backwards and turn
    p3dx.move(-1.0,1.0)
    p3dx.tilt(-0.3)
    p3dx.sleep(4)
    p3dx.move(0,0)

In [14]:
p3dx.tilt(-0.3)

try:
    while True:
        b = blue_balls()
        for ball in b:
            get_ball(ball[1],ball[2])
            grip_ball()
            get_target()
            place_ball()
        p3dx.move(-1,1)
except KeyboardInterrupt:
    p3dx.move(0,0)


getting ball..
done aligning.approaching ball
done approaching.getting closer
gripping ball..
getting target..
target centered
approaching target
getting closer to target..
target centered
approaching target
Placing ball..
moving back
getting ball..
done aligning.approaching ball
done approaching.getting closer
gripping ball..
getting target..
target centered
approaching target
getting closer to target..
target centered
approaching target
Placing ball..
moving back
getting ball..
done aligning.approaching ball
done approaching.getting closer
gripping ball..
getting target..
target centered
approaching target
getting closer to target..
target centered
approaching target
Placing ball..
moving back
getting ball..
done aligning.approaching ball
done approaching.getting closer
gripping ball..
getting target..
target centered
approaching target
getting closer to target..
target centered
approaching target
Placing ball..
moving back
getting ball..
done aligning.approaching ball
done approachi

---
#### Try-a-Bot: an open source guide for robot programming
Developed by:
[![Robotic Intelligence Lab @ UJI](img/logo/robinlab.png "Robotic Intelligence Lab @ UJI")](http://robinlab.uji.es)

Sponsored by:
<table>
<tr>
<td style="border:1px solid #ffffff ;">
<a href="http://www.ieee-ras.org"><img src="img/logo/ras.png"></a>
</td>
<td style="border:1px solid #ffffff ;">
<a href="http://www.cyberbotics.com"><img src="img/logo/cyberbotics.png"></a>
</td>
<td style="border:1px solid #ffffff ;">
<a href="http://www.theconstructsim.com"><img src="img/logo/theconstruct.png"></a>
</td>
</tr>
</table>

Follow us:
<table>
<tr>
<td style="border:1px solid #ffffff ;">
<a href="https://www.facebook.com/RobotProgrammingNetwork"><img src="img/logo/facebook.png"></a>
</td>
<td style="border:1px solid #ffffff ;">
<a href="https://www.youtube.com/user/robotprogrammingnet"><img src="img/logo/youtube.png"></a>
</td>
</tr>
</table>